# Vector Store Index

In this guide, we show how to use the vector store index with different vector store
implementations.  
 
From how to get started with few lines of code with the default
in-memory vector store with default query configuration, to using a custom hosted vector
store, with advanced settings such as metadata filters.


### Construct vector store and index
**Default**

By default, `VectorStoreIndex` uses a in-memory `SimpleVectorStore`
that's initialized as part of the default storage context.

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# Load documents and build index
documents = SimpleDirectoryReader('../../examples/data/paul_graham').load_data()
index = VectorStoreIndex.from_documents(documents)


**Custom vectors store**

You can use a custom vector store (in this case `PineconeVectorStore`) as follows:

In [ ]:
import pinecone
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores import PineconeVectorStore

# init pinecone
pinecone.init(api_key="<api_key>", environment="<environment>")
pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

# construct vector store and customize storage context
storage_context = StorageContext.from_defaults(
    vector_store = PineconeVectorStore(pinecone.Index("quickstart"))
)

# Load documents and build index
documents = SimpleDirectoryReader('../../examples/data/paul_graham').load_data()
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)


For more examples of how to initialize different vector stores, 
see [Vector Store Integrations](/how_to/integrations/vector_stores.md).


### Query
**Default**  

You can start querying by getting the default query engine:

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")


**Configure standard query setting**  

To configure query settings, you can directly pass it as
keyword args when building the query engine: 

In [ ]:
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

query_engine = index.as_query_engine(
    similarity_top_k=3,
    vector_store_query_mode="default",
    filters=MetadataFilters(filters=[
        ExactMatchFilter(key='name', value='paul graham'),
    ]),
    alpha=None,
    doc_ids=None,
)
response = query_engine.query("what did the author do growing up?")

Note that metadata filtering is applied against metadata specified in `Node.extra_info`.

Alternatively, if you are using the lower-level compositional API:

In [ ]:
from llama_index.indices.query.response_synthesis import ResponseSynthesizer
from llama_index.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine

# build retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
    vector_store_query_mode="default",
    filters=[
        ExactMatchFilter(key='name', value='paul graham')
    ],
    alpha=None,
    doc_ids=None,
)

# build query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=ResponseSynthesizer.from_args()
)

# query
response = query_engine.query("what did the author do growing up?")

**Configure vector store specific keyword arguments**  

You can customize keyword arguments unique to a specific vector store implementation as well by passing in `vector_store_kwargs`

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=3,
    # only works for pinecone
    vector_store_kwargs={
        "filter": {'name': 'paul graham'},
    },
)
response = query_engine.query("what did the author do growing up?")

**Use an auto retriever**

You can also use an LLM to automatically decide query setting for you! 
Right now, we support automatically setting exact match metadata filters and top k parameters.

In [ ]:
from llama_index.indices.query.response_synthesis import ResponseSynthesizer
from llama_index.indices.vector_store.retrievers import VectorIndexAutoRetriever
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine
from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo


vector_store_info = VectorStoreInfo(
    content_info='brief biography of celebrities',
    metadata_info=[
        MetadataInfo(
            name='category', 
            type='str', 
            description='Category of the celebrity, one of [Sports, Entertainment, Business, Music]'),
        MetadataInfo(name='country', type='str', description='Country of the celebrity, one of [United States, Barbados, Portugal]'),
    ]
)

# build retriever
retriever = VectorIndexAutoRetriever(index, vector_store_info=vector_store_info)

# build query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=ResponseSynthesizer.from_args()
)

# query
response = query_engine.query("Tell me about two celebrities from United States")